# Download and Install Requisisites

## Clone Repository

In [ ]:
!git clone https://github.com/SugarcaneAI/ThereGoesMangoesDataset.git

## Install Ultralytics PyPackage to access the YOLO algorithm 

In [ ]:
%pip install ultralytics

# Actual Training

## Import necessary libraries

* Python path processing utility library `pathlib` (imported as `pl`)
* The YOLO `YOLO` class class to access the `YOLO` algorithm from Ultralytics `ultralytics`

In [ ]:
import pathlib as pl

from ultralytics import YOLO

from google.colab import files

import locale

## Global Parameters

* Dataset Root folder `PARAM_DATASET_ROOT`
* Dataset YAML file `PARAM_DATASET_YAML`

In [ ]:
PARAM_DATASET_ROOT = pl.Path("/content/ThereGoesMangoesDataset")
PARAM_DATASET_YAML = PARAM_DATASET_ROOT.joinpath("yolo_mango_detection.yaml")

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

## Training Loop

Proceed with actual training.

In [ ]:
for mname, msize in [("nano", "n"), ("small", "s"), ("medium", "m"), ("large", "l"), ("xl", "x")]:
  for imgsz in [640, 1024]:
    try:
      # Load model
      model = YOLO(f"yolov8{msize}.pt")
      model.train(
          data=PARAM_DATASET_YAML,
          epochs=500,
          batch=10,
          imgsz=imgsz,
          project="mangoes",
          name=f"{mname}-{imgsz}",
          exist_ok=True,
      )

      # validate
      model.val(
          split="test",
          name=f"{mname}-{imgsz}-val",
          exist_ok=True,
      )
    except Exception as exc:
      print(f"Caught exception: {exc}. Aborting.")
      continue

# Exporting

## Export to TF `tensorflow` formats and download save files 

In [ ]:
for mname, msize in [("nano", "n"), ("small", "s"), ("medium", "m"), ("large", "l"), ("xl", "x")]:
  for imgsz in [256, 512, 640, 1024]:
    path = pl.Path.cwd().joinpath("mangoes").joinpath(f"{mname}-{imgsz}/weights/best.pt")

    if not path.exists():
      continue

    # load in model
    model = YOLO(path)

    # Export to TFLite
    model.export(format="tflite")

    # Export to TF Saved Model
    model.export(format="saved_model", keras=True)

    # create ZIP file (for downloading later)
    !zip -r /content/{mname}-{imgsz}.zip /content/mangoes/{mname}-{imgsz}

    # wait for ZIP creation
    fpath = pl.Path(f"/content/{mname}-{imgsz}.zip")
    while not fpath.exists():
      continue

    # download file
    files.download(fpath)

gdfbdcghjf